# 사용자 activity 로그를 Hive table 로 제공하기 위한 Spark Application

## spark 버전 및 연결 확인

In [1]:
!spark-submit --version
//!spark-shell --version
//!spark-sql --version

24/05/27 19:36:37 WARN Utils: Your hostname, Hwikeunui-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 172.29.40.132 instead (on interface en0)


24/05/27 19:36:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.3.2
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 16.0.1
Branch HEAD
Compiled by user liangchi on 2023-02-10T19:57:40Z
Revision 5103e00c4ce5fcc4264ca9c4df12295d42557af6
Url https://github.com/apache/spark
Type --help for more information.



Intitializing Scala interpreter ...

Spark Web UI available at http://172.29.40.132:4041
SparkContext available as 'sc' (version = 3.3.2, master = local[*], app id = local-1716806200986)
SparkSession available as 'spark'


In [2]:
sc.version

res1: String = 3.3.2


In [3]:
spark.version

res2: String = 3.3.2


## 데이터 처리

In [6]:
val SPARK_HOME=sys.env.get("SPARK_HOME").mkString 
val FILE_PATH = ".//data//2019-Oct.csv"
val PATH=FILE_PATH
val df = spark.read.csv(PATH)
df.show()


+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|                 _c0|       _c1|       _c2|                _c3|                 _c4|     _c5|    _c6|      _c7|                 _c8|
+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
|2019-10-01 00:00:...|      view|  44600062|2103807459595387724|                null|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:...|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|  33.20|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:...|      view|  17200506|2053013559792632471|furniture.living_...|    null| 543.10|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:...|      view|   1307067|205301355892021719

SPARK_HOME: String = /Users/keun/anaconda3/lib/python3.10/site-packages/pyspark
FILE_PATH: String = .//data//2019-Oct.csv
PATH: String = .//data//2019-Oct.csv
df: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 7 more fields]
